In [1]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
inter_df = pd.read_csv('./국토교통부_건축물대장_표제부/bdinfo_baseline_maindata_inter_checkpoint_2.csv')
inter_df['사용승인_일'] = inter_df['사용승인_일'].astype('Int64')

c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
print(inter_df.shape)
inter_df.head()

(894005, 51)


,관리_건축물대장_PK,대장_구분,대장_종류,외필지_수,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조,기타_구조,주_용도,기타_용도,지붕,기타_지붕,높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),사용승인_일,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소,총세대수,대장구분명,기준년월_최초사용승인일,기준년월_최근사용승인일,주소별_최초생성일자,주소별_최근생성일자,사용승인_년도,주소별_기준년월_총세대수합
0,11440-21639,집합,표제부,0,177.62,105.300,59.28,393.130,385.210,216.87,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,(철근)콘크리트 평스라브,12.00,4,1,0,0,0,0.0,393.130,0,0.0,0,0.0,0,0.00,4,46.0,20040323,NaN,0.0,0,NaN,0,NaN,0,20101230,201412,서울특별시 마포구 서교동 484-16,4,일반,20040323.0,20040323.0,20101230,20170209,2004.0,4
1,11590-100182663,집합,표제부,1,389.48,232.420,59.67,947.740,659.890,169.43,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,(철근)콘크리트,11.26,4,1,0,0,0,0.0,947.740,0,0.0,0,0.0,8,287.85,2,11.5,20080826,NaN,0.0,0,NaN,0,NaN,0,20110119,201412,서울특별시 동작구 상도동 302-28,10,일반,20080826.0,20080826.0,20110119,20190530,2008.0,10
2,11320-100182241,집합,표제부,0,208.00,81.085,38.98,268.165,268.165,128.93,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,(철근)콘크리트,10.40,4,0,0,0,0,0.0,268.165,0,0.0,0,0.0,3,0.00,1,0.0,20080721,NaN,0.0,0,NaN,0,NaN,0,20110202,201412,서울특별시 도봉구 쌍문동 380-24,4,일반,20080721.0,20080721.0,20110202,20190530,2008.0,4
3,11440-32174,집합,표제부,0,314.00,180.380,57.45,627.880,627.880,199.96,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,평스라브,13.30,5,0,0,0,0,0.0,627.880,0,0.0,0,0.0,7,80.50,1,11.5,20050901,NaN,0.0,0,NaN,0,NaN,0,20110125,201412,서울특별시 마포구 성산동 200-260,8,일반,20050901.0,20050901.0,20110125,20171205,2005.0,8
4,11740-100179671,집합,표제부,0,0.00,564.010,0.00,6401.190,6401.190,0.00,철근콘크리트구조,철근콘크리트구조,공동주택,공동주택(아파트),(철근)콘크리트,(철근)콘크리트,45.60,14,0,3,0,0,0.0,6401.190,0,0.0,0,0.0,0,0.00,0,0.0,20090311,NaN,0.0,0,NaN,0,NaN,0,20110122,201412,서울특별시 강동구 강일동 674-0,78,일반,20090311.0,20090311.0,20100918,20200428,2009.0,1001


In [5]:
inter_df['대장_종류'].value_counts()

표제부      844880
일반건축물     49125
Name: 대장_종류, dtype: int64

In [6]:
inter_df['대장_구분'].value_counts()

집합    844880
일반     49125
Name: 대장_구분, dtype: int64

In [7]:
wbdinfo_df = pd.read_csv('./국토교통부_건축물대장_총괄표제부/wbdinfo_baseline_2.csv')
wbdinfo_df.shape

c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(43449, 44)

In [8]:
wbdinfo_df.head()

,관리_건축물대장_PK,대장_구분,대장_종류,신_구_대장_구분,건물_명,외필지_수,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),주_용도,기타_용도,주_건축물_수,부속_건축물_수,부속_건축물_면적(㎡),총_주차_수,옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),사용승인_일,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소,총세대수,대장구분명,기준년월_최초사용승인일,기준년월_최근사용승인일,주소별_최초생성일자,주소별_최근생성일자,주소별_기준년월_총세대수합
0,11650-2,집합,총괄표제부,구대장,NaN,0,359.3,162.78,45.30,493.41,325.56,90.61,공동주택,공동주택,2,0,0.0,2,0,0.0,0,0.0,0,0.00,2,30.00,NaN,NaN,0.0,0,NaN,0,NaN,0,20100417,201412,서울특별시 서초구 방배동 911-29,8,일반,NaN,NaN,20100417,20171118,8
1,11650-2,집합,총괄표제부,신대장,트라움하우스 5,0,4321.4,1855.67,42.94,12660.24,6129.15,141.83,공동주택,공동주택(연립주택),3,0,0.0,0,0,0.0,0,0.0,110,4522.93,0,0.00,20030416.0,NaN,0.0,0,NaN,0,NaN,0,20100417,201412,서울특별시 서초구 서초동 1496-26,18,일반,20030416.0,20030416.0,20100417,20161220,18
2,11650-3,집합,총괄표제부,신대장,방배동 갤러리하우스,1,884.0,413.10,46.73,3695.03,2346.09,265.39,공동주택,공동주택(아파트),2,0,0.0,28,0,0.0,0,0.0,28,1305.56,0,0.00,20030514.0,NaN,0.0,0,NaN,0,NaN,0,20100417,201412,서울특별시 서초구 방배동 1-7,16,일반,20030514.0,20030514.0,20100417,20171118,16
3,11650-4,집합,총괄표제부,구대장,NaN,1,380.5,190.20,49.98,570.60,380.40,99.97,공동주택,공동주택,2,0,0.0,3,0,0.0,0,0.0,0,0.00,3,41.25,NaN,NaN,0.0,0,NaN,0,NaN,0,20100417,201412,서울특별시 서초구 반포동 577-73,12,일반,NaN,NaN,20100417,20100417,12
4,11650-,집합,총괄표제부,신대장,방배동다세대주택,0,493.0,292.30,59.29,1234.89,1095.26,222.16,공동주택,"공동주택,다세대주택",2,0,0.0,12,0,0.0,0,0.0,12,0.00,0,0.00,20030925.0,NaN,0.0,0,NaN,0,NaN,0,20100417,201412,서울특별시 서초구 방배동 919-10,15,일반,20030925.0,20030925.0,20100417,20171118,15


In [9]:
bdinfo_df = inter_df.copy()

In [10]:
inter_df = bdinfo_df[bdinfo_df['지번주소'].isin(wbdinfo_df['지번주소'].unique())]
inter_df.shape

(195314, 51)

In [11]:
inter_df.shape

(195314, 51)

In [12]:
bdinfo_df.shape

(894005, 51)

In [13]:
bdinfo_df.drop(inter_df.index, inplace=True)
bdinfo_df.shape

(698691, 51)

In [14]:
bdinfo_df['대장_구분'].value_counts()

집합    657957
일반     40734
Name: 대장_구분, dtype: int64

In [15]:
bdinfo_df.head()

,관리_건축물대장_PK,대장_구분,대장_종류,외필지_수,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조,기타_구조,주_용도,기타_용도,지붕,기타_지붕,높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),사용승인_일,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소,총세대수,대장구분명,기준년월_최초사용승인일,기준년월_최근사용승인일,주소별_최초생성일자,주소별_최근생성일자,사용승인_년도,주소별_기준년월_총세대수합
0,11440-21639,집합,표제부,0,177.62,105.300,59.28,393.130,385.210,216.87,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,(철근)콘크리트 평스라브,12.00,4,1,0,0,0,0.0,393.130,0,0.0,0,0.0,0,0.00,4,46.0,20040323,NaN,0.0,0,NaN,0,NaN,0,20101230,201412,서울특별시 마포구 서교동 484-16,4,일반,20040323.0,20040323.0,20101230,20170209,2004.0,4
1,11590-100182663,집합,표제부,1,389.48,232.420,59.67,947.740,659.890,169.43,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,(철근)콘크리트,11.26,4,1,0,0,0,0.0,947.740,0,0.0,0,0.0,8,287.85,2,11.5,20080826,NaN,0.0,0,NaN,0,NaN,0,20110119,201412,서울특별시 동작구 상도동 302-28,10,일반,20080826.0,20080826.0,20110119,20190530,2008.0,10
2,11320-100182241,집합,표제부,0,208.00,81.085,38.98,268.165,268.165,128.93,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,(철근)콘크리트,10.40,4,0,0,0,0,0.0,268.165,0,0.0,0,0.0,3,0.00,1,0.0,20080721,NaN,0.0,0,NaN,0,NaN,0,20110202,201412,서울특별시 도봉구 쌍문동 380-24,4,일반,20080721.0,20080721.0,20110202,20190530,2008.0,4
3,11440-32174,집합,표제부,0,314.00,180.380,57.45,627.880,627.880,199.96,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,평스라브,13.30,5,0,0,0,0,0.0,627.880,0,0.0,0,0.0,7,80.50,1,11.5,20050901,NaN,0.0,0,NaN,0,NaN,0,20110125,201412,서울특별시 마포구 성산동 200-260,8,일반,20050901.0,20050901.0,20110125,20171205,2005.0,8
5,11380-28561,집합,표제부,0,167.90,100.600,59.92,390.730,390.730,232.72,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택(8세대),(철근)콘크리트,평 스라브,12.50,5,0,0,0,0,0.0,390.730,0,0.0,0,0.0,0,0.00,6,0.0,20020708,NaN,0.0,0,NaN,0,NaN,0,20100825,201412,서울특별시 은평구 응암동 88-17,8,일반,20020708.0,20020708.0,20100825,20100825,2002.0,8


In [16]:
bdinfo_df['주소별_기준년월_총세대수합'].describe()

count    698691.000000
mean         23.397575
std         163.127815
min           0.000000
25%           6.000000
50%           8.000000
75%          11.000000
max        5154.000000
Name: 주소별_기준년월_총세대수합, dtype: float64

In [17]:
bdinfo_df['주소별_기준년월_총세대수합'] = bdinfo_df[['지번주소', '기준년월', '총세대수']].groupby(['지번주소', '기준년월']).transform('sum')['총세대수']

In [19]:
bdinfo_df['주소별_기준년월_총세대수합'].describe()

count    698691.000000
mean         23.397575
std         163.127815
min           0.000000
25%           6.000000
50%           8.000000
75%          11.000000
max        5154.000000
Name: 주소별_기준년월_총세대수합, dtype: float64

In [20]:
zero_df = bdinfo_df[bdinfo_df['주소별_기준년월_총세대수합'] == 0]
zero_df.shape

(4674, 51)

In [21]:
bdinfo_df.drop(zero_df.index, inplace=True)
bdinfo_df.shape

(694017, 51)

In [22]:
bdinfo_df['주소별_기준년월_총세대수합'].describe()

count    694017.000000
mean         23.555151
std         163.664864
min           1.000000
25%           6.000000
50%           8.000000
75%          11.000000
max        5154.000000
Name: 주소별_기준년월_총세대수합, dtype: float64

In [23]:
max_df = bdinfo_df[bdinfo_df['주소별_기준년월_총세대수합'] == bdinfo_df['주소별_기준년월_총세대수합'].max()]
max_df.shape

(147, 51)

In [24]:
max_df.head()

,관리_건축물대장_PK,대장_구분,대장_종류,외필지_수,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조,기타_구조,주_용도,기타_용도,지붕,기타_지붕,높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),사용승인_일,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소,총세대수,대장구분명,기준년월_최초사용승인일,기준년월_최근사용승인일,주소별_최초생성일자,주소별_최근생성일자,사용승인_년도,주소별_기준년월_총세대수합
7540,11500-3364,집합,표제부,0,0.0,4562.88,0.0,11194.53,5200.65,0.0,철근콘크리트구조,철근콘크리트조,공동주택,아파트,(철근)콘크리트,스라브,0.0,15,1,0,0,6,0.0,4562.88,0,0.0,0,0.0,0,0.0,0,0.0,19931025,NaN,0.0,0,NaN,0,NaN,0,20140902,201412,서울특별시 강서구 방화동 839-0,238,일반,19901231.0,19931025.0,20111117,20171205,1993.0,5154
13462,11500-3358,집합,표제부,0,0.0,7137.60,0.0,12458.42,8412.45,0.0,철근콘크리트구조,철근콘크리트조,공동주택,아파트,(철근)콘크리트,스라브,0.0,15,1,0,0,6,0.0,7137.60,0,0.0,0,0.0,0,0.0,0,0.0,19931025,NaN,0.0,0,NaN,0,NaN,0,20140902,201412,서울특별시 강서구 방화동 839-0,238,일반,19901231.0,19931025.0,20111117,20171205,1993.0,5154
13463,11500-3367,집합,표제부,0,0.0,5042.88,0.0,11674.53,5680.65,0.0,철근콘크리트구조,철근콘크리트조,공동주택,아파트,(철근)콘크리트,스라브,0.0,15,1,0,0,6,0.0,5042.88,0,0.0,0,0.0,0,0.0,0,0.0,19931025,NaN,0.0,0,NaN,0,NaN,0,20140902,201412,서울특별시 강서구 방화동 839-0,298,일반,19901231.0,19931025.0,20111117,20171205,1993.0,5154
25371,11500-3376,집합,표제부,0,0.0,5930.88,0.0,12562.53,6568.65,0.0,철근콘크리트구조,철근콘크리트조,공동주택,아파트,(철근)콘크리트,스라브,0.0,15,1,0,0,6,0.0,5930.88,0,0.0,0,0.0,0,0.0,0,0.0,19931025,NaN,0.0,0,NaN,0,NaN,0,20140902,201412,서울특별시 강서구 방화동 839-0,358,일반,19901231.0,19931025.0,20111117,20171205,1993.0,5154
27535,11500-3366,집합,표제부,0,0.0,5042.88,0.0,11674.53,5680.65,0.0,철근콘크리트구조,철근콘크리트조,공동주택,아파트,(철근)콘크리트,스라브,0.0,15,1,0,0,6,0.0,5042.88,0,0.0,0,0.0,0,0.0,0,0.0,19931025,NaN,0.0,0,NaN,0,NaN,0,20140902,201412,서울특별시 강서구 방화동 839-0,298,일반,19901231.0,19931025.0,20111117,20171205,1993.0,5154


In [25]:
bdinfo_df.to_csv('./국토교통부_건축물대장_표제부/bdinfo_except_checkpoint_2.csv', index=False)